<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/naver_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def page_to_df(self, page):
    url = "https://finance.naver.com/item/sise_day.nhn?code=" + self.code + "&page=" + str(page)
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})
    data_dict = {}
    for i in range(10):
      try: 
        spans = tr[i].find_all('span')
        lines = [span.get_text() for span in spans]
        lines = list(map(lambda x: x.replace(',',''), lines))

        objdate = datetime.datetime.strptime(lines[0], '%Y.%m.%d')
        data_dict[objdate] = lines[1:]
      except:
        break
      
    df = pd.DataFrame.from_dict(data_dict).transpose()
    df.columns = ['Close','_','Open',
                        'High','Low','Volume']
    df = df.drop(columns="_")

    return df

  def get_data(self, start_page:int=1, end_page:int=26):
    df = self.page_to_df(start_page)
    # max 9999
    # 1 page = 10 days
    for i in range(start_page, end_page): 
      try: 
        if (any(self.page_to_df(i).index != self.page_to_df(i-1).index)):
          df = df.append(self.page_to_df(i))
          print('.', end = '')
          if (i%100==0):
            print(i)
        else:
          break
      except:
        df = df.append(self.page_to_df(i))
    df = df.sort_index(axis = 0) 
    for i in ['Close', 'Open', 'High', 'Low', 'Volume']:
      df[i] = df[i].astype(int)
    return df

In [2]:
from multiprocessing import Pool

stock = Stock('005930')

with Pool(500) as p:
    li = p.starmap(stock.get_data, [(x,x) for x in range(500,0,-1)])

In [3]:
df = pd.concat(li)

In [4]:
df

,Close,Open,High,Low,Volume
2001-02-14,199000,196000,202000,195000,394201
2001-02-15,206500,210000,210500,205000,698680
2001-02-16,212500,207500,215500,206500,927555
2001-02-19,209500,209000,210500,205000,457630
2001-02-20,212000,209500,214000,209000,363573
...,...,...,...,...,...
2021-04-29,81700,82400,82500,81500,20000973
2021-04-30,81500,81900,82100,81500,18673197
2021-05-03,81700,81000,82400,81000,15710336
2021-05-04,82600,81900,82600,81800,12532550
